# **Run the cells below to prepare the data (REQUIRED)**

In [ ]:
import time
import dat
import pandas as pd
import numpy as np
%pip install openpyxl
import openpyxl

In [ ]:
print("Creating model to compute DAT...")

model = dat.Model("glove_100_3_polish.txt", "words.txt")

print("Finished creating model.")

**place the prepared xlsx file in the project's folder and change the filename accordingly**

In [ ]:
file_name = 'dat-data.xlsx'

In [ ]:
try:
    
    df_upload = pd.read_excel(file_name, header=None)
    df_upload = df_upload.fillna(value=' ')
    data = df_upload.values.tolist()
    print('upload successful')
    
except IndexError:
    print('Error: upload a file first')
    
except FileNotFoundError:
    print(f'Error: Could not find file: {file_name}.')

**Run cell below to obtain the DAT score**

In [ ]:
for i in data:
    
    try:
        x = model.dat(i)
        if x != None:
            print(x.round(2))
        else:
            print(x)
    
    except TypeError:
        print(i)

**Run cell below to obtain words not found in model**

In [ ]:
inv_words_list = []

for i in data:
    
    try:
        x = model.invalid_words(i)
        if len(x) < 1:
            continue
        print(', '.join(x))
        inv_words_list.append(x)

    except TypeError:
        print(i)
    
if len(inv_words_list) < 1:
    print("no invalid words")

**Run cell below to obtain a html file with distances by pairs**

In [ ]:
columns = ['W1-W2','W1-W3', 'W1-W4', 'W1-W5', 'W1-W6', 'W1-W7', 
                'W2-W3', 'W2-W4', 'W2-W5', 'W2-W6', 'W2-W7', 
                'W3-W4', 'W3-W5', 'W3-W6', 'W3-W7', 
                'W4-W5', 'W4-W6', 'W4-W7', 
                'W5-W6', 'W5-W7', 
                'W6-W7']


date = time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())

word_pairs = []
for i in data:
    x = model.dat(i, all_words=True)
    if x != None:
        word_pairs.append(x)
    else:
        word_pairs.append([''])
        

df = pd.DataFrame(word_pairs, columns=columns)
df.to_csv(f'dat_distances{date}.csv')

**Run cells below to obtain dat score and matrix for all given words (if 7 minimum were given)**

In [ ]:
for i in data:
    x = model.full_dat(i)
    if x != None:
        print(x.round(2))
    else:
        print(x)
        

In [ ]:
all_columns = ['W1-W2','W1-W3','W1-W4','W1-W5','W1-W6','W1-W7','W7-W8','W1-W9','W1-W10', 
                'W2-W3','W2-W4','W2-W5','W2-W6','W2-W7','W2-W8','W2-W9','W2-W10', 
                'W3-W4','W3-W5','W3-W6','W3-W7','W3-W8','W3-W9','W3-W10', 
                'W4-W5', 'W4-W6','W4-W7','W4-W8','W4-W9','W4-W10', 
                'W5-W6', 'W5-W7','W5-W8','W5-W9','W5-W10', 
                'W6-W7','W6-W8','W6-W9','W6-W10',
                'W7-W8','W7-W9','W7-W10', 
                'W8-W9','W8-W10', 
                'W9-W10'
              ]

date = time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())

word_pairs = []
for i in data:
    x = model.full_dat(i,1)
    if x != None:
        word_pairs.append(x)
    else:
        word_pairs.append([''])
        
df = pd.DataFrame(word_pairs, columns=all_columns)
df.to_csv(f'all_dat_distances{date}.csv')
